In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Load the training data (assuming 'train.csv' is in the current directory)
df = pd.read_csv("train.csv")

# Drop the 'Id' column as it's not a feature
df = df.drop("Id", axis=1)

# Separate features and target
X = df.drop("Cover_Type", axis=1).values
y = df["Cover_Type"].values - 1  # Adjust classes to 0-6 for CrossEntropyLoss

# Convert to PyTorch tensors
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).long()

# Custom Dataset class
class ForestDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset and dataloader
dataset = ForestDataset(X, y)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the neural network model
class ForestCoverModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(54, 128)  # 54 input features
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 7)    # 7 output classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, loss function, and optimizer
model = ForestCoverModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop (train for 10 epochs as an example; adjust as needed)
for epoch in range(10):
    for batch in loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Function to predict forest cover type for a new 30m x 30m patch
# Input: a list or array of 54 features (in the order of the columns, excluding Id and Cover_Type)
def predict_forest_cover(input_data):
    input_tensor = torch.from_numpy(np.array(input_data)).float()
    if len(input_tensor.shape) == 1:
        input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension if single sample
    with torch.no_grad():
        output = model(input_tensor)
    pred = torch.argmax(output, dim=1) + 1  # Adjust back to 1-7 classes
    return pred.numpy()  # Return as numpy array; for single prediction, pred[0]

# Example usage (replace with actual feature values):
# example_input = [2596, 51, 3, 258, 0, 510, 221, 232, 148, 6279, 1, 0, 0, 0, ...]  # 54 values
# prediction = predict_forest_cover(example_input)
# print(f"Predicted Cover Type: {prediction}")

Epoch 1, Loss: 2.2135448455810547
Epoch 2, Loss: 0.8710484504699707
Epoch 3, Loss: 0.8472139239311218
Epoch 4, Loss: 1.03107750415802
Epoch 5, Loss: 0.7146432995796204
Epoch 6, Loss: 0.4262605309486389
Epoch 7, Loss: 0.9204602837562561
Epoch 8, Loss: 0.7234874963760376
Epoch 9, Loss: 0.7262040972709656
Epoch 10, Loss: 0.49577397108078003
